In [ ]:
import os
import pytesseract
from PIL import Image

folder_path = "/Users/ajahedi/Downloads/Raw_tissue plots/IWP5/IWP5Reg01/no"
  # Replace with your folder path

# List all files in the directory
all_files = os.listdir(folder_path)

for filename in all_files:
    image_path = os.path.join(folder_path, filename)
    try:
        img = Image.open(image_path)
        # rest of the processing code...
    except (Image.UnidentifiedImageError, IOError):
        print(f"Skipping non-image file: {filename}")
        continue
   
    # Open the image
    img = Image.open(image_path)

    # Extract text using OCR
    extracted_text = pytesseract.image_to_string(img)
    
    # Get the first line from the extracted text
    first_line = extracted_text.split('\n')[0].strip()
    
    # Construct the new file name
    new_filename = first_line + os.path.splitext(filename)[1]
    new_filepath = os.path.join(folder_path, new_filename)

    # Rename the file
    os.rename(image_path, new_filepath)

print("Files renamed successfully!")


In [14]:
import os
import pytesseract
from PIL import Image

base_folder_path = "/Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & Papers/qc folder/Stitching/Raw_tissue plots/IWP5/IWP5Reg06"
subdirs = [d for d in os.listdir(base_folder_path) if os.path.isdir(os.path.join(base_folder_path, d))]

for subdir in subdirs:
    folder_path = os.path.join(base_folder_path, subdir)

    # List all files in the directory
    all_files = os.listdir(folder_path)

    for filename in all_files:
        image_path = os.path.join(folder_path, filename)
        try:
            img = Image.open(image_path)
        except (Image.UnidentifiedImageError, IOError):
            print(f"Skipping non-image file: {filename}")
            continue

        # Extract text using OCR
        extracted_text = pytesseract.image_to_string(img)
        
        # Get the first line from the extracted text
        first_line = extracted_text.split('\n')[0].strip()
        
        # Construct the new file name
        new_filename = first_line + os.path.splitext(filename)[1]
        new_filepath = os.path.join(folder_path, new_filename)

        # Rename the file
        os.rename(image_path, new_filepath)

print("Files renamed successfully!")


Files renamed successfully!


In [16]:
import numpy as np
import matplotlib.pyplot as plt
import os

def calculate_2dft(input):
    ft = np.fft.ifftshift(input)
    ft = np.fft.fft2(ft)
    return np.fft.fftshift(ft)

def calculate_2dift(input):
    ift = np.fft.ifftshift(input)
    ift = np.fft.ifft2(ift)
    ift = np.fft.fftshift(ift)
    return ift.real

def calculate_distance_from_centre(coords, centre):
    return np.sqrt((coords[0] - centre) ** 2 + (coords[1] - centre) ** 2)

def find_symmetric_coordinates(coords, centre):
    return (centre + (centre - coords[0]), centre + (centre - coords[1]))

folder_path = "/Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & Papers/qc folder/Stitching/IWP5Reg01/high/"

# Extract the desired folder name from the path
folder_name = os.path.basename(os.path.dirname(folder_path))
match = re.search(r'IWP\d+Reg\d+', folder_path)
if match:
    plot_title_prefix = match.group()
else:
    plot_title_prefix = folder_name

# List all files in the directory
all_files = os.listdir(folder_path)

# Filter out only PNG files
png_files = [f for f in all_files if f.endswith('.png')]

for image_filename in png_files:
    full_image_path = os.path.join(folder_path, image_filename)
    image = plt.imread(full_image_path)
    image = image[:, :, :3].mean(axis=2)  # Convert to grayscale
    
    # Fourier transform
    ft = calculate_2dft(image)

    array_size = len(image)
    centre = int((array_size - 1) / 2)
    coords_left_half = ((x, y) for x in range(array_size) for y in range(centre+1))
    coords_left_half = sorted(coords_left_half, key=lambda x: calculate_distance_from_centre(x, centre))

    # Reconstruct image up to term 50
    rec_image = np.zeros(image.shape)
    individual_grating = np.zeros(image.shape, dtype="complex")

    for idx, coords in enumerate(coords_left_half, start=1):
        if idx > 50:
            break
        symm_coords = find_symmetric_coordinates(coords, centre)
        individual_grating[coords] = ft[coords]
        individual_grating[symm_coords] = ft[symm_coords]
        rec_grating = calculate_2dift(individual_grating)
        rec_image += rec_grating
        individual_grating[coords] = 0
        individual_grating[symm_coords] = 0

    # Display the reconstructed image after the 50th term with the title
    plt.figure()
    plt.imshow(rec_image, cmap="gray")
    plt.title(f"{plot_title_prefix} - {image_filename} (Term 50)")
    plt.axis("off")
    plt.show()


NameError: name 're' is not defined

In [58]:
#Improving OCR text detection not to overwrite the images.
import os
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter

def preprocess_image_for_ocr(img_path):
    """Preprocess the image to improve OCR results."""
    img = Image.open(img_path)
    
    # Convert to grayscale
    img = img.convert('L')
    
    # Enhance the contrast of the image
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(2)
    
    # Apply a median filter to reduce noise
    img = img.filter(ImageFilter.MedianFilter(3))
    
    return img

base_folder_path = "/Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & Papers/qc folder/Stitching/Raw_tissue plots/IWP5/"

# Get all subfolders within IWP11
regions = [d for d in os.listdir(base_folder_path) if os.path.isdir(os.path.join(base_folder_path, d))]

for region in regions:
    region_path = os.path.join(base_folder_path, region)
    subdirs = [d for d in os.listdir(region_path) if os.path.isdir(os.path.join(region_path, d))]

    for subdir in subdirs:
        folder_path = os.path.join(region_path, subdir)

        # List all files in the directory
        all_files = os.listdir(folder_path)

        for filename in all_files:
            image_path = os.path.join(folder_path, filename)
            try:
                img = preprocess_image_for_ocr(image_path)
            except (Image.UnidentifiedImageError, IOError):
                print(f"Skipping non-image file: {filename}")
                continue

            # Extract text using OCR
            extracted_text = pytesseract.image_to_string(img)
            
            # Get the first line from the extracted text
            first_line = extracted_text.split('\n')[0].strip()
            
            # If first_line is empty, skip renaming for this file
            if not first_line:
                print(f"OCR did not recognize text for {filename}. Skipping renaming.")
                continue
            
            # Construct the new file name
            new_filename = first_line + os.path.splitext(filename)[1]
            new_filepath = os.path.join(folder_path, new_filename)

            # Ensure filename is unique
            counter = 1
            while os.path.exists(new_filepath):
                new_filename = first_line + f"_{counter}" + os.path.splitext(filename)[1]
                new_filepath = os.path.join(folder_path, new_filename)
                counter += 1

            # Rename the file
            os.rename(image_path, new_filepath)

print("Files renamed successfully!")


KeyboardInterrupt: 

In [132]:
#create list of stitching vs non-stitching 
import os
import pandas as pd

base_folder_path = "/Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & Papers/qc folder/Stitching/Raw_tissue plots/IWR10/"
max_regions = 10  # Assuming a maximum of 10 regions; adjust as needed

# Create an empty dataframe to store results for all regions
df_all = pd.DataFrame()

for region_number in range(1, max_regions + 1):
    region_folder = os.path.join(base_folder_path, f"IWR10Reg{region_number:02}")
    
    # Check if region folder exists
    if not os.path.exists(region_folder):
        continue

    # Collect data: folder names and their markers
    data = {}
    subdirs = [d for d in os.listdir(region_folder) if os.path.isdir(os.path.join(region_folder, d))]

    for subdir in subdirs:
        folder_path = os.path.join(region_folder, subdir)
        filenames = [f for f in os.listdir(folder_path) if f.endswith('.png')]  # directly using filenames
        data[subdir] = filenames

    # Construct the table for the current region
    df = pd.DataFrame(columns=set(val for sublist in data.values() for val in sublist))  # unique filenames as columns
    for folder, filenames in data.items():
        df.loc[folder] = [1 if filename in filenames else 0 for filename in df.columns]

    # Append the dataframe of the current region to the main dataframe with the modified index
    df_all = df_all.append(df.set_index(df.index + '_' + os.path.basename(region_folder)))

# Save the final dataframe to a CSV file
output_path = os.path.join(base_folder_path, "table_of_contents_all_regions.csv")
df_all.to_csv(output_path)

print(f"Table of contents saved successfully at: {output_path}")


Table of contents saved successfully at: /Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & Papers/qc folder/Stitching/Raw_tissue plots/IWR10/table_of_contents_all_regions.csv


In [29]:

import os
import pandas as pd

base_folder_path = "/Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & Papers/qc folder/Stitching/Raw_tissue plots/IWR10/"

# Collect data: folder names and their PNG files
data = {}
all_files = set()  # to store all unique PNG filenames across all folders

# Using os.walk to traverse directories and subdirectories
for dirpath, dirnames, filenames in os.walk(base_folder_path):
    # Filtering out non-PNG files
    png_files = [f for f in filenames if f.endswith('.png')]
    if png_files:
        relative_dirpath = os.path.relpath(dirpath, base_folder_path)
        data[relative_dirpath] = png_files
        all_files.update(png_files)

# Construct the table
df = pd.DataFrame(0, index=list(all_files), columns=list(data.keys()))  # initialize with zeros
for folder, filenames in data.items():
    df.loc[filenames, folder] = 1

# Save the dataframe to a CSV file
df.to_csv(os.path.join(base_folder_path, "table_of_contents.csv"))

print("Table of contents saved successfully!")


Table of contents saved successfully!


In [4]:
#renaming for one folder region
import os
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter

def preprocess_image_for_ocr(img_path):
    """Preprocess the image to improve OCR results."""
    img = Image.open(img_path)
    
    # Convert to grayscale
    img = img.convert('L')
    
    # Enhance the contrast of the image
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(2)
    
    # Apply a median filter to reduce noise
    img = img.filter(ImageFilter.MedianFilter(3))
    
    return img

base_folder_path = "/Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & Papers/computational files/qc/Stitching/Raw_tissue plots/IWP5/IWP5Reg01"
subdirs = [d for d in os.listdir(base_folder_path) if os.path.isdir(os.path.join(base_folder_path, d))]

for subdir in subdirs:
    folder_path = os.path.join(base_folder_path, subdir)

    # List all files in the directory
    all_files = os.listdir(folder_path)

    for filename in all_files:
        image_path = os.path.join(folder_path, filename)
        try:
            img = preprocess_image_for_ocr(image_path)
        except (Image.UnidentifiedImageError, IOError):
            print(f"Skipping non-image file: {filename}")
            continue

        # Extract text using OCR
        extracted_text = pytesseract.image_to_string(img)
        
        # Get the first line from the extracted text
        first_line = extracted_text.split('\n')[0].strip()
        
        # If first_line is empty, skip renaming for this file
        if not first_line:
            print(f"OCR did not recognize text for {filename}. Skipping renaming.")
            continue
        
        # Construct the new file name
        new_filename = first_line + os.path.splitext(filename)[1]
        new_filepath = os.path.join(folder_path, new_filename)

        # Ensure filename is unique
        counter = 1
        while os.path.exists(new_filepath):
            new_filename = first_line + f"_{counter}" + os.path.splitext(filename)[1]
            new_filepath = os.path.join(folder_path, new_filename)
            counter += 1

        # Rename the file
        os.rename(image_path, new_filepath)

print("Files renamed successfully!")


Skipping non-image file: .DS_Store
Files renamed successfully!


In [133]:
import os
import pandas as pd

base_folder_path = "/Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & Papers/qc folder/Stitching/Raw_tissue plots/"

# Collect all subfolders
subdirs = [d for d in os.listdir(base_folder_path) if os.path.isdir(os.path.join(base_folder_path, d))]

# Create an empty dataframe to store all combined data
df_combined = pd.DataFrame()

for subdir in subdirs:
    csv_file = os.path.join(base_folder_path, subdir, "table_of_contents_all_regions.csv")
    
    # Check if the CSV file exists
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file, index_col=0)
        df_combined = pd.concat([df_combined, df], axis=0)  # Append the data to the master dataframe

# Save the combined dataframe to a new CSV file
output_path = os.path.join(base_folder_path, "combined_table_of_contents.csv")
df_combined.to_csv(output_path)

print(f"Combined table of contents saved successfully at: {output_path}")


Combined table of contents saved successfully at: /Users/ajahedi/OneDrive - Inside MD Anderson/CODEX Resources & Papers/qc folder/Stitching/Raw_tissue plots/combined_table_of_contents.csv
